In [1]:
import numpy as np, pandas as pd

In [2]:
data_dir = '/home/j/DATA/KEN/KIHBS/2005_2006/'

weekly_expend = 'KEN_KIHBS_2005_2006_SECTION_I_WEEKLY_EXPEND_Y2017M03D03.DTA'
ident_file = 'KEN_KIHBS_2005_2006_SECTION_A_IDENTIFICATION_Y2017M03D03.DTA'
housing_file = 'KEN_KIHBS_2005_2006_SECTION_G_HOUSING_Y2017M03D03.DTA'

In [3]:
%%time

r = pd.read_stata(data_dir + weekly_expend,
                  chunksize=100_000, preserve_dtypes=False)

df = pd.DataFrame()
for t in r:
    print('.', flush=True, end=' ')
    df = df.append(t)
df.shape

. . . CPU times: user 527 ms, sys: 170 ms, total: 698 ms
Wall time: 696 ms


(276115, 25)

In [4]:
%%time

r = pd.read_stata(data_dir + ident_file,
                  chunksize=100_000, preserve_dtypes=False)

ident = pd.DataFrame()
for t in r:
    print('.', flush=True, end=' ')
    ident = ident.append(t)
ident.shape

r = pd.read_stata(data_dir + housing_file,
                  chunksize=100_000, preserve_dtypes=False)

hh = pd.DataFrame()
for t in r:
    print('.', flush=True, end=' ')
    hh = hh.append(t)
hh.shape

. . CPU times: user 176 ms, sys: 3.22 ms, total: 179 ms
Wall time: 176 ms


(13074, 24)

In [82]:
var_map = {
    'cooking_oil':905,
    'maize_grain':103,
    'green_maize':104,
    'maize_flour_loose':105,
    'maize_flour_sifted':106,
    'wheat_grain':107,
    'wheat_flour':108
}

In [6]:
# how many unique elements when i try to key?
df[['id_clust','id_hh']].drop_duplicates().shape, ident[['id_clust','id_hh','hhsize','weight']].drop_duplicates().shape, hh[['id_clust','id_hh','weights']].drop_duplicates().shape

((13158, 2), (13212, 4), (13074, 3))

In [7]:
compare_w = ident.merge(hh[['id_clust','id_hh','weights']], on = ['id_clust','id_hh'], how = 'left')
compare_w.shape, ident.shape

((13212, 16), (13212, 15))

In [8]:
compare_w =compare_w.rename(columns = {'weight':'ident_weight','weights':'hh_weight'})

In [9]:
df = df.merge(compare_w, on = ['id_clust','id_hh'], how = 'left')

In [16]:
df = df.rename(columns = {'i02':'var_name'})

In [18]:
df['var_name'] = ['item_{}'.format(i) for i in df.var_name]

In [20]:
df.head()

,id_clust,id_hh,i01,var_name,i03q,i03u,i03k,i04,i04k,i05q,...,dmonth,dyear,hhsize,a09,a11,a12,a13,filter,ident_weight,hh_weight
0,1,1,yes,item_101,7.0,kilogramme,280.0,7.000,280.00,0.0,...,12.0,2005.0,2,embu,no,NaN,NaN,Households used for poverty estimation,863.6,863.6
1,1,1,yes,item_103,1.0,debe,300.0,1.000,300.00,0.0,...,12.0,2005.0,2,embu,no,NaN,NaN,Households used for poverty estimation,863.6,863.6
2,1,1,yes,item_105,0.0,0,0.0,0.000,0.00,0.0,...,12.0,2005.0,2,embu,no,NaN,NaN,Households used for poverty estimation,863.6,863.6
3,1,1,yes,item_106,10.0,kilogramme,300.0,10.000,300.00,0.0,...,12.0,2005.0,2,embu,no,NaN,NaN,Households used for poverty estimation,863.6,863.6
4,1,1,yes,item_108,6.0,kilogramme,210.0,0.006,0.21,0.0,...,12.0,2005.0,2,embu,no,NaN,NaN,Households used for poverty estimation,863.6,863.6


In [56]:
keep_cols = ['id_clust','id_hh','weights','ident_weight','hh_weight','hhsize','var_name'] + [i for i in df.columns if i.startswith('i0')]
consum = df[keep_cols].pivot_table(index = ['id_clust','id_hh','weights','ident_weight','hh_weight','hhsize'], columns = 'var_name', values = 'i07q', fill_value = 0)
purch = df[keep_cols].pivot_table(index = ['id_clust','id_hh','weights','ident_weight','hh_weight','hhsize'], columns = 'var_name', values = 'i03q', fill_value = 0)

In [90]:
df[keep_cols].head()

,id_clust,id_hh,weights,ident_weight,hh_weight,hhsize,var_name,i01,i03q,i03u,...,i05au,i05ap,i05ak,i06q,i06u,i06p,i06k,i07q,i07u,i07k
0,1,1,863.6,863.6,863.6,2,item_101,yes,7.0,kilogramme,...,0,0.0,0.0,0.0,0,0.0,0.0,7.0,kilogramme,0.0
1,1,1,863.6,863.6,863.6,2,item_103,yes,1.0,debe,...,0,0.0,0.0,0.0,0,0.0,0.0,1.0,debe,0.0
2,1,1,863.6,863.6,863.6,2,item_105,yes,0.0,0,...,0,0.0,0.0,10.0,kilogramme,30.0,300.0,10.0,kilogramme,0.0
3,1,1,863.6,863.6,863.6,2,item_106,yes,10.0,kilogramme,...,0,0.0,0.0,0.0,0,0.0,0.0,10.0,kilogramme,0.0
4,1,1,863.6,863.6,863.6,2,item_108,yes,6.0,kilogramme,...,0,0.0,0.0,0.0,0,0.0,0.0,6.0,kilogramme,0.0


In [57]:
consum = consum.reset_index()
purch = purch.reset_index()

In [112]:
[i for i in df.columns if i.startswith('i04')]

['i04', 'i04k']

In [119]:
consum_purch = df[keep_cols].pivot_table(index = ['id_clust','id_hh','weights','ident_weight','hh_weight','hhsize'], columns = 'var_name', values = 'i04', fill_value = 0)
consum_purch = consum_purch.reset_index()

In [40]:
var_map

{'cooking_oil': 905,
 'maize_grain': 103,
 'green_maize': 104,
 'maize_flour_loose': 105,
 'maize_flour_sifted': 106}

In [42]:
np.arange(103,107)

array([103, 104, 105, 106])

In [58]:
maize_cols = ['item_{}'.format(i) for i in range(103,107)]
maize_cols

['item_103', 'item_104', 'item_105', 'item_106']

In [59]:
consum['maize_any'] = consum[maize_cols].sum(axis=1)
purch['maize_any'] = purch[maize_cols].sum(axis=1)

In [120]:
consum_purch['maize_any'] = consum_purch[maize_cols].sum(axis=1)

In [122]:
num = (consum[consum.maize_any > 0].weights * consum[consum.maize_any > 0].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any maize: {num / denom}')

num = (purch[purch.maize_any > 0].weights * purch[purch.maize_any > 0].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any maize: {num / denom}')

num = (consum_purch[consum_purch.maize_any > 0].weights * consum_purch[consum_purch.maize_any > 0].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased maize: {num / denom}')

prop that consumed any maize: 0.9666808082878499
prop that purchased any maize: 0.6303421150083685
prop that consumed any purchased maize: 0.6299038313935422


In [125]:
label = 'maize_grain'
var = 'item_{}'.format(var_map[label])

num = (consum[consum[var] > 0].weights * consum[consum[var] > 0].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label}: {num / denom}')

num = (purch[purch[var] > 0].weights * purch[purch[var] > 0].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label}: {num / denom}')

num = (consum_purch[consum_purch[var] > 0].weights * consum_purch[consum_purch[var] > 0].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label}: {num / denom}')

prop that consumed any maize_grain: 0.5767273292716147
prop that purchased any maize_grain: 0.3666360575275327
prop that consumed any purchased maize_grain: 0.36630923067298854


In [126]:
label = 'green_maize'
var = 'item_{}'.format(var_map[label])

num = (consum[consum[var] > 0].weights * consum[consum[var] > 0].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label}: {num / denom}')

label = 'green_maize'
var = 'item_{}'.format(var_map[label])

num = (purch[purch[var] > 0].weights * purch[purch[var] > 0].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label}: {num / denom}')

num = (consum_purch[consum_purch[var] > 0].weights * consum_purch[consum_purch[var] > 0].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label}: {num / denom}')

prop that consumed any green_maize: 0.1764635429890018
prop that purchased any green_maize: 0.07973369888140701
prop that consumed any purchased green_maize: 0.07938033720597068


In [129]:
label1 = 'maize_flour_loose'
label2 = 'maize_flour_sifted'
var1 = 'item_{}'.format(var_map[label1])
var2 = 'item_{}'.format(var_map[label2])

num = (consum[(consum[var1] > 0) | (consum[var2] > 0)].weights * consum[(consum[var1] > 0) | (consum[var2] > 0)].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label1} or {label2}: {num / denom}')

num = (purch[(purch[var1] > 0) | (purch[var2] > 0)].weights * purch[(purch[var1] > 0) | (purch[var2] > 0)].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label1} or {label2}: {num / denom}')

num = (consum_purch[(consum_purch[var1] > 0) | (consum_purch[var2] > 0)].weights * consum_purch[(consum_purch[var1] > 0) | (consum_purch[var2] > 0)].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label1} or {label2}: {num / denom}')

prop that consumed any maize_flour_loose or maize_flour_sifted: 0.8527429066632093
prop that purchased any maize_flour_loose or maize_flour_sifted: 0.3668524364177676
prop that consumed any purchased maize_flour_loose or maize_flour_sifted: 0.366618773375352


In [130]:
label1 = 'green_maize'
label2 = 'maize_grain'
var1 = 'item_{}'.format(var_map[label1])
var2 = 'item_{}'.format(var_map[label2])

num = (consum[(consum[var1] > 0) | (consum[var2] > 0)].weights * consum[(consum[var1] > 0) | (consum[var2] > 0)].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label1} or {label2}: {num / denom}')

num = (purch[(purch[var1] > 0) | (purch[var2] > 0)].weights * purch[(purch[var1] > 0) | (purch[var2] > 0)].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label1} or {label2}: {num / denom}')

num = (consum_purch[(consum_purch[var1] > 0) | (consum_purch[var2] > 0)].weights * consum_purch[(consum_purch[var1] > 0) | (consum_purch[var2] > 0)].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label1} or {label2}: {num / denom}')

prop that consumed any green_maize or maize_grain: 0.6790428353370863
prop that purchased any green_maize or maize_grain: 0.42032512946082307
prop that consumed any purchased green_maize or maize_grain: 0.4200016787909995


In [131]:
label = 'maize_flour_loose'
var = 'item_{}'.format(var_map[label])

num = (consum[consum[var] > 0].weights * consum[consum[var] > 0].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label}: {num / denom}')

var = 'item_{}'.format(var_map[label])

num = (purch[purch[var] > 0].weights * purch[purch[var] > 0].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label}: {num / denom}')

num = (consum_purch[consum_purch[var] > 0].weights * consum_purch[consum_purch[var] > 0].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label}: {num / denom}')

prop that consumed any maize_flour_loose: 0.5803442066114469
prop that purchased any maize_flour_loose: 0.09623597297553474
prop that consumed any purchased maize_flour_loose: 0.0961859828010402


In [132]:
label = 'maize_flour_sifted'
var = 'item_{}'.format(var_map[label])

num = (consum[consum[var] > 0].weights * consum[consum[var] > 0].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label}: {num / denom}')

var = 'item_{}'.format(var_map[label])

num = (purch[purch[var] > 0].weights * purch[purch[var] > 0].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label}: {num / denom}')

num = (consum_purch[consum_purch[var] > 0].weights * consum_purch[consum_purch[var] > 0].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label}: {num / denom}')

prop that consumed any maize_flour_sifted: 0.32923164927773646
prop that purchased any maize_flour_sifted: 0.28107918474448035
prop that consumed any purchased maize_flour_sifted: 0.2808617256821425


In [133]:
label = 'cooking_oil'
var = 'item_{}'.format(var_map[label])

num = (consum[consum[var] > 0].weights * consum[consum[var] > 0].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label}: {num / denom}')

var = 'item_{}'.format(var_map[label])

num = (purch[purch[var] > 0].weights * purch[purch[var] > 0].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label}: {num / denom}')

num = (consum_purch[consum_purch[var] > 0].weights * consum_purch[consum_purch[var] > 0].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label}: {num / denom}')

prop that consumed any cooking_oil: 0.16173560365456752
prop that purchased any cooking_oil: 0.09404242231779887
prop that consumed any purchased cooking_oil: 0.09394942994385358


In [134]:
label = 'wheat_flour'
var = 'item_{}'.format(var_map[label])

num = (consum[consum[var] > 0].weights * consum[consum[var] > 0].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label}: {num / denom}')

var = 'item_{}'.format(var_map[label])

num = (purch[purch[var] > 0].weights * purch[purch[var] > 0].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label}: {num / denom}')

num = (consum_purch[consum_purch[var] > 0].weights * consum_purch[consum_purch[var] > 0].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label}: {num / denom}')

prop that consumed any wheat_flour: 0.3657739686584146
prop that purchased any wheat_flour: 0.309480451939186
prop that consumed any purchased wheat_flour: 0.3093679447218349


In [135]:
label = 'wheat_grain'
var = 'item_{}'.format(var_map[label])

num = (consum[consum[var] > 0].weights * consum[consum[var] > 0].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label}: {num / denom}')

var = 'item_{}'.format(var_map[label])

num = (purch[purch[var] > 0].weights * purch[purch[var] > 0].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label}: {num / denom}')

num = (consum_purch[consum_purch[var] > 0].weights * consum_purch[consum_purch[var] > 0].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label}: {num / denom}')

prop that consumed any wheat_grain: 0.008047758111279679
prop that purchased any wheat_grain: 0.006733555281750999
prop that consumed any purchased wheat_grain: 0.006733555281750999


In [136]:
label1 = 'wheat_grain'
label2 = 'wheat_flour'
var1 = 'item_{}'.format(var_map[label1])
var2 = 'item_{}'.format(var_map[label2])

num = (consum[(consum[var1] > 0) | (consum[var2] > 0)].weights * consum[(consum[var1] > 0) | (consum[var2] > 0)].hhsize).sum()
denom = (consum.weights * consum.hhsize).sum()

print(f'prop that consumed any {label1} or {label2}: {num / denom}')


num = (purch[(purch[var1] > 0) | (purch[var2] > 0)].weights * purch[(purch[var1] > 0) | (purch[var2] > 0)].hhsize).sum()
denom = (purch.weights * purch.hhsize).sum()

print(f'prop that purchased any {label1} or {label2}: {num / denom}')

num = (consum_purch[(consum_purch[var1] > 0) | (consum_purch[var2] > 0)].weights * consum_purch[(consum_purch[var1] > 0) | (consum_purch[var2] > 0)].hhsize).sum()
denom = (consum_purch.weights * consum_purch.hhsize).sum()

print(f'prop that consumed any purchased {label1} or {label2}: {num / denom}')

prop that consumed any wheat_grain or wheat_flour: 0.3701767510119157
prop that purchased any wheat_grain or wheat_flour: 0.3141792620013736
prop that consumed any purchased wheat_grain or wheat_flour: 0.3140667547840225
